In [1]:
import csv
import json
import pandas as pd
import ast

In [2]:


# Function to convert JSON to CSV
def json_to_csv(json_file, csv_file):
    # Read JSON data
    with open(json_file, mode='r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Write data to CSV file
    with open(csv_file, mode='w', encoding='utf-8', newline='') as f:
        csv_writer = csv.DictWriter(f, fieldnames=data[0].keys())
        csv_writer.writeheader()
        csv_writer.writerows(data)

# Example usage
json_to_csv('D:/Freelance Projects/scrapers/amazon/json/mens_clothes.json', 'D:/Freelance Projects/scrapers/amazon/json/mens_shoes.csv')


In [3]:
merged_df4 = pd.read_csv('D:/Freelance Projects/scrapers/amazon/json/mens_shoes.csv')

In [4]:
# Assuming your original dataframe is named 'merged_df4'

# Calculate number of reviews by each user
reviews_count = merged_df4.groupby('user_id').size().reset_index(name='reviews_count')

# Calculate average length of reviews by each user
merged_df4['review_length'] = merged_df4['text'].apply(lambda x: len(x) if isinstance(x, str) else 1)  # Assuming 'text' column contains review text
# merged_df4['review_length'] = [len(data) for data in merged_df4['text'] if isinstance(data, str) else 1]  # Assuming 'text' column contains review text
# print(type(merged_df4['review_length']))
avg_review_length = merged_df4.groupby('user_id')['review_length'].mean().reset_index(name='avg_review_length')

# Calculate average rating by each user
avg_rating = merged_df4.groupby('user_id')['review_rating'].mean().reset_index(name='avg_rating')

# Calculate average helpfulness votes got by each user on their reviews
avg_helpful_votes = merged_df4.groupby('user_id')['helpful_votes'].mean().reset_index(name='avg_helpful_votes')

# Merge all the calculated features into a single dataframe
user_insights_df1 = reviews_count.merge(avg_review_length, on='user_id').merge(avg_rating, on='user_id').merge(avg_helpful_votes, on='user_id')

# Display the resulting dataframe
# print(user_insights_df1)
user_insights_df1

merged_df4 = pd.merge(merged_df4,user_insights_df1 , on='user_id', how='inner')
merged_df4.head()


,user_name,user_id,product_title,average_rating,total_rating,product_category,features,description,product_asin,review_title,...,total_helpful_votes,similarity_text_description,similarity_text_features,similarity_text_categories,sentiment_scores,review_length,reviews_count,avg_review_length,avg_rating,avg_helpful_votes
0,Amazon Customer,AHVXWQNICMFYFD7UEUBEUUZCOSGQ,Polaroid PLD 2053/S Oval Sunglasses,4.3,1289,"Clothing, Shoes & Jewelry, Women, Accessories,...","{'Origin': 'Imported', 'Country of Origin': 'C...","Not rx-able, Metal frame; TAC (Tri Acetate Cel...",B0D5G5P938,"Damaged, way too small.",...,11,0.233507,0.0,0.098852,-0.2773,304,1,304.000000,1.000000,1.000000
1,Aldo Pacheco,AEEI3DMFYC5YXT73RER4BAJDCXMQ,Chef Works Cool Vent Baseball Cap,4.5,1406,"Clothing, Shoes & Jewelry, Men, Accessories, H...","{'Fabric type': '100% Polyester', 'Care instru...","All over exclusive cool vent fabric, Adjustabl...",B0D8JG71WL,It doesn’t last for too much time,...,5,0.399774,0.0,0.180899,-0.5256,144,1,144.000000,2.000000,1.000000
2,Robb Horstman,AG3CBWAWBJKRVDIQ3GR35KZBUKHA,Nautica Men's Bold Fashion and Dress Leather B...,4.4,995,"Clothing, Shoes & Jewelry, Men, Accessories, B...","{'Fabric type': '99% Canvas, 1% Leather', 'Car...",Men's genuine leather and canvas belt for ever...,B0D6CQ4XTQ,This belt is not leather,...,0,0.346264,0.0,0.145786,-0.7263,192,1,192.000000,1.000000,0.000000
3,jennifer dutzy,AFWXN6UISR5DOPD7LL4VOIIKG23A,Polaroid PLD 6014/S Men's Sunglasses,4.5,732,"Clothing, Shoes & Jewelry, Women, Accessories,...","{'Fabric type': 'Rubber', 'Origin': 'Imported'...","style: Pld6014/S, Case included, Unisex Rubber...",B07BXZSCR5,They sent the wrong glasses and color,...,0,0.469551,0.0,0.155990,0.0000,250,1,250.000000,1.000000,0.000000
4,Kimberly,AF3SF4KVGUIR6T2YLQIRR2IKCFCQ,Trump Merchandise Donald Trump Hat with Hair M...,4.7,2752,"Clothing, Shoes & Jewelry, Men, Accessories, H...","{'Fabric type': 'Polyester,Cotton', 'Care inst...","METERIAL: 65% Cotton 35% Polyester Metal, INCL...",B08Y8D25Y3,Awesome hat!! Trump 2024,...,125,0.748875,0.0,0.184968,0.8770,133,3,132.333333,4.666667,0.333333


In [5]:
# Function to convert CSV to JSON
def csv_to_json(csv_file, json_file):
    data = []

    # Read CSV file
    with open(csv_file, mode="r", encoding="utf-8") as f:
        csv_reader = csv.DictReader(f)

        # Convert each row into a dictionary and add to the data list
        for row in csv_reader:
            row["features"] = ast.literal_eval(row["features"])
            row["average_rating"] = float(row["average_rating"])
            row["total_rating"] = int(row["total_rating"])
            row["verified_purchase"] = bool(row["verified_purchase"])
            row["review_rating"] = float(row["review_rating"])
            row["helpful_votes"] = int(row["helpful_votes"])
            row["total_helpful_votes"] = int(row["total_helpful_votes"])
            row["similarity_text_description"] = float(
                row["similarity_text_description"]
            )
            row["similarity_text_features"] = float(row["similarity_text_features"])
            row["similarity_text_categories"] = float(row["similarity_text_categories"])
            row["sentiment_scores"] = float(row["sentiment_scores"])
            data.append(row)

    # Write data to JSON file
    with open(json_file, mode="w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


# Example usage
csv_to_json(
    "D:/Freelance Projects/scrapers/amazon/json/mens_shoes.csv",
    "D:/Freelance Projects/scrapers/amazon/json/mens_shoes.json",
)